In [25]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from pandas import DataFrame

In [2]:
mact = pd.read_csv('MACT_location_capacity.csv').drop(columns='Unnamed: 0')

mact.loc[:,'State'] = mact.FacilityID.str[0:2]

mact.loc[:,'FacilityID'] = mact.FacilityID.str[2:]

mact.loc[:,'FacilityID'] = mact.apply(lambda x: re.sub(r"(\w)([A-Z])", r"\1 \2", x['FacilityID']),axis=1)

mact.loc[:,'mactID'] = mact.index + 1


#list of unique FacilityIDs in mact data set
mact_fac = mact['FacilityID'].unique()

In [3]:
tier = pd.read_csv('Tier_overall_1.csv').rename(columns={'COUNTY_FIPS':'FIPS'})

tier = tier[(tier['NAICS']>=311111)&
            (tier['NAICS']<=339999)].drop_duplicates(subset=tier.columns.difference(['COUNTY']))

tier.loc[:,'tierID'] = tier.index + 1


#create dictionary of unique tier FACILITY_NAMEs with list of mact FacilityIDs
tier_fac = tier['FACILITY_NAME'].unique()

tier_fac_df = DataFrame(tier_fac,columns=['FACILITY_NAME'])

tier_fac_df.loc[:,'Facility_ID'] = tier_fac_df.apply(lambda x: process.extractOne(x['FACILITY_NAME'],mact_fac),axis=1)

tier_fac_df['Facility_ID'] = tier_fac_df.Facility_ID.astype(str).str[2:-6]

fac_dict = tier_fac_df.set_index('FACILITY_NAME')['Facility_ID'].to_dict()

In [4]:
#add column with the mact FacilityID that was the closest match
tier.loc[:,'FacilityID'] = tier.apply(lambda x: fac_dict[x['FACILITY_NAME']],axis=1)

#merge based on FacilityID and FIPS
tier_mact = tier.merge(mact,how="inner",on=['FacilityID','FIPS'])


In [5]:
#compare capacitiy values, and keep only matching capacity values
tier_mact_matches = tier_mact.loc[(tier_mact['INPUT_HEAT_CAPACITY']==tier_mact['Capacity (mmBtu/hr)']) | 
                          (tier_mact['AGGR_HIGH_HEAT_CAPACITY']==tier_mact['Capacity (mmBtu/hr)'])].copy()

In [18]:
#remove duplicate ghgrp_tier entries by keeping only the first mact entry that matched to it
tier_mact_matches.drop_duplicates(subset='tierID',
                                  keep="first", inplace=True)

In [21]:
#identify entries that were not matched and create combined dataframe

tierID_matches = tier_mact_matches['tierID'].unique()
mactID_matches = tier_mact_matches['mactID'].unique()

tier_nm = tier.loc[~tier['tierID'].isin(tierID_matches)].drop(columns='FacilityID')
mact_nm = mact.loc[~mact['mactID'].isin(mactID_matches)].rename(columns={'FIPS':'FIPS_m','NAICS':'NAICS_sub'})
                   
tier_mact_nonmatch = pd.concat([tier_nm, mact_nm], ignore_index=True,sort=False)

In [22]:
tier_mact_matches.to_csv('tier_mact_matches.csv',index=False)
tier_mact_nonmatch.to_csv('tier_mact_nonmatch.csv',index=False)